# Read Graph inputs

Path: /fast_scratch/atlas_images/v01-45

In [1]:
import numpy as np
import uproot

In [2]:
from tqdm import tqdm

In [3]:
!ls -a

.   .git		README.md	       efn_Dilia.ipynb
..  .ipynb_checkpoints	ReadGraphInputs.ipynb


In [4]:
# Just read /fast_scratch/atlas_images/v01-45/delta/user.angerami.24409108.OutputStream._000257.root

In [5]:
filename = '/fast_scratch/atlas_images/v01-45/delta/user.angerami.24409108.OutputStream._000257.root'
f = uproot.open(filename)

In [6]:
### content and type
#f.classnames()  
f.keys() #content

['EventTree;1', 'CellGeo;1']

In [7]:
### Use the form file['key'] to access an object inside a file.
ev_tree =  f['EventTree;1']
cell_geo_tree = f['CellGeo;1']

## Understanding the input with uproot

In [8]:
### Branches
#ev_tree.keys()
cell_geo_tree.keys()

['cell_geo_ID',
 'cell_geo_sampling',
 'cell_geo_eta',
 'cell_geo_phi',
 'cell_geo_rPerp',
 'cell_geo_deta',
 'cell_geo_dphi',
 'cell_geo_volume',
 'cell_geo_sigma']

In [9]:
cell_geo_tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
cell_geo_ID          | std::vector<uint64_t>    | AsJagged(AsDtype('>u8'), he...
cell_geo_sampling    | std::vector<uint16_t>    | AsJagged(AsDtype('>u2'), he...
cell_geo_eta         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_phi         | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_rPerp       | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_deta        | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_dphi        | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_volume      | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_sigma       | std::vector<float>       | AsJagged(AsDtype('>f4'), he...


In [10]:
cell_geo_tree.arrays()['cell_geo_ID'] #Connects the event tree with the cell tree

<Array [... 1284492592, 1284493104]] type='1 * var * uint64'>

In [11]:
cell_geo_tree["cell_geo_sigma"].array(library="pd") # library="np"

entry  subentry
0      0           49.457161
       1           49.457161
       2           49.457161
       3           49.457161
       4           49.457161
                     ...    
       187645      20.233513
       187646      10.413343
       187647      10.957963
       187648      12.509910
       187649      11.231500
Length: 187650, dtype: float32

In [12]:
cluster = ev_tree.arrays(["cluster_E","cluster_Eta","cluster_Phi"])

In [13]:
cluster["cluster_Eta"]

<Array [[-0.414, -0.446, ... -0.112, -0.202]] type='20000 * var * float32'>

In [14]:
## track information & cluster information
#ev_tree.keys(filter_name="track*")
ev_tree.keys(filter_name="*cluster*")

['cluster_E',
 'cluster_E_LCCalib',
 'cluster_Pt',
 'cluster_Eta',
 'cluster_Phi',
 'cluster_nCells',
 'cluster_ENG_CALIB_TOT',
 'cluster_ENG_CALIB_OUT_T',
 'cluster_ENG_CALIB_DEAD_TOT',
 'cluster_EM_PROBABILITY',
 'cluster_HAD_WEIGHT',
 'cluster_OOC_WEIGHT',
 'cluster_DM_WEIGHT',
 'cluster_CENTER_MAG',
 'cluster_FIRST_ENG_DENS',
 'cluster_CENTER_LAMBDA',
 'cluster_ISOLATION',
 'cluster_ENERGY_DigiHSTruth',
 'cluster_cell_ID',
 'cluster_cell_E',
 'cluster_hitsTruthIndex',
 'cluster_hitsTruthE']

In [15]:
#cluster_cell_ID ->>> compact ID for geometry lookup     SAME AS  cell_geo_ID in CellGeo tree
#cluster_cell_E  ->>> cell energy in GeV
ev_tree.keys(filter_name="*cluster_cell*")

['cluster_cell_ID', 'cluster_cell_E']

In [16]:
## The second argument is a cut, or filter on entries. ### trackZ0 is always negative
ev_tree.arrays(['trackZ0'],"trackZ0>0")

<Array [{trackZ0: []}, ... {trackZ0: []}] type='20000 * {"trackZ0": var * float32}'>

In [17]:
ev_tree.arrays("chi2DOF", aliases={"chi2DOF": "trackChiSquared / trackNumberDOF"})

<Array [{chi2DOF: [1.14], ... 0.917]}] type='20000 * {"chi2DOF": var * float64}'>

## Cluster information

In [18]:
n_events = ev_tree.num_entries
print("number of events:", n_events)

number of events: 20000


In [19]:
cluster_var = ['cluster_EM_PROBABILITY', 'cluster_HAD_WEIGHT', 'cluster_OOC_WEIGHT',
               'cluster_DM_WEIGHT', 'cluster_CENTER_MAG', 'cluster_FIRST_ENG_DENS', 
               'cluster_CENTER_LAMBDA', 'cluster_ISOLATION',  
              ]

In [42]:
ev_tree["cluster_EM_PROBABILITY"].array(library="pd") 

entry  subentry
0      0           0.895813
       1           0.513963
       2           0.897644
1      0           0.000337
       1           0.213792
                     ...   
19997  5           0.587252
       6           0.468654
19999  0           0.000246
       1           0.097961
       2           0.007718
Length: 74452, dtype: float32

In [21]:
cluster_var_dict_musig = {}

In [22]:
for ivar in cluster_var : 
    var = np.concatenate(ev_tree[ivar].array(library='np'))
    cluster_var_dict_musig[ivar] =  {'mean' : var.mean(), 'std' : var.std()}

In [23]:
#### Array with mu and sigma of the clustat variables cluster_var
cluster_var_dict_musig

{'cluster_EM_PROBABILITY': {'mean': 0.18505172, 'std': 0.2732016},
 'cluster_HAD_WEIGHT': {'mean': 1.0895879, 'std': 0.10427206},
 'cluster_OOC_WEIGHT': {'mean': 1.4219106, 'std': 0.43727043},
 'cluster_DM_WEIGHT': {'mean': 1.1682752, 'std': 0.31482273},
 'cluster_CENTER_MAG': {'mean': 3573.3145, 'std': 1125.9248},
 'cluster_FIRST_ENG_DENS': {'mean': 1.5478874e-05, 'std': 8.048178e-05},
 'cluster_CENTER_LAMBDA': {'mean': 715.20215, 'std': 766.0365},
 'cluster_ISOLATION': {'mean': 0.78983563, 'std': 0.21683465}}

## Get Cells positions

CellGeo (cell_geo_tree) just have one entry. 

In [24]:
cell_geo_tree["cell_geo_sampling"].array(library="np") # sampling layer stored as a short int

array([array([ 6,  6,  6, ..., 17, 17, 17], dtype=uint16)], dtype=object)

In [25]:
cell_geo_tree["cell_geo_volume"].array(library="np") # 3D volume of cell in mm^3

array([array([1697610.  , 1697610.  , 1697610.  , ..., 1241209.5 ,  773987.6 ,
               466650.06], dtype=float32)                                     ],
      dtype=object)

In [26]:
cell_geo_tree["cell_geo_sigma"].array(library="np") #  noise value from noise tool (in MeV)

array([array([49.45716 , 49.45716 , 49.45716 , ..., 10.957963, 12.50991 ,
              11.2315  ], dtype=float32)                                 ],
      dtype=object)

* ### get_all_cells

In [27]:
### Cell position information formatted on numpy arrays 
rperp = cell_geo_tree['cell_geo_rPerp'].array(library='np')[0]      # cylindrical radius $\sqrt{ x^2+y^2} 
cell_eta = cell_geo_tree['cell_geo_eta'].array(library='np')[0]
cell_phi = cell_geo_tree['cell_geo_phi'].array(library='np')[0]

cell_deta = cell_geo_tree['cell_geo_deta'].array(library='np')[0]      # delta eta and dphi of the cell ?? 
cell_dphi = cell_geo_tree['cell_geo_dphi'].array(library='np')[0]

cell_theta = 2*np.arctan( np.exp(-cell_eta) )

In [28]:
#### x,y,z cell position
cell_x = rperp*np.cos(cell_phi)
cell_y = rperp*np.sin(cell_phi)
cell_z = rperp/np.tan(cell_theta)

cell_positions = np.column_stack([cell_x,cell_y,cell_z])

Instead of saving in cell_positions the $x,y,z$ coordinates of the cell -> save rapidity $y$, $\phi$?? 

OR

Match with cell ID to the event tree and then save [energy, rapidity, phi] of the cell

Decorrelate angular with energy info ?  $y= \frac{1}{2}\ln(\frac{E+p_z}{E-p_z})$ 

In [29]:
cell_geo_ID = cell_geo_tree['cell_geo_ID'].array(library='np')[0]     # to connect with cluster_cell_ID

In [30]:
# Dictionaries with cell ID as key. 
#the entries can be paired with the zip() function
id_to_position = {c_id : pos  for c_id,pos in zip(cell_geo_ID,cell_positions)}
id_to_deta     = {c_id : deta for c_id,deta in zip(cell_geo_ID,cell_deta)}
id_to_dphi     = {c_id : dphi for c_id,dphi in zip(cell_geo_ID,cell_dphi)}

In [31]:
id_to_position[740294656]

array([  616.83887 ,    33.279976, -3970.4194  ], dtype=float32)

* ### get_single_event

In [48]:
print("number of events:", ev_tree.num_entries )
print("**FIRST EVENT**")
print("Number of clusters:  ", ev_tree['nCluster'].array()[0]) 
print("Number of cells per cluster:  ", ev_tree['cluster_nCells'].array()[0]) 
print("Number of cells in the first cluster:  ", ev_tree['cluster_nCells'].array()[0][0] ) 
print("?????Number of Cell ID in the first cluster", len(ev_tree['cluster_cell_ID'].array()[0][0]))   ## Selection criteria?
print("?????Cell Energies in the first cluster", len(ev_tree['cluster_cell_E'].array()[0][0]))

number of events: 20000
**FIRST EVENT**
Number of clusters:   3
Number of cells per cluster:   [53, 54, 40]
Number of cells in the first cluster:   53
?????Number of Cell ID in the first cluster 38
?????Cell Energies in the first cluster 38


In [33]:
#Event index:
event_idx = 0

In [34]:
ev_tree["cluster_cell_ID"].array(library="pd")

0        ((759177244, 759177242, 759177246, 759176732, ...
1        ((1141524224, 1141507840, 1141540608, 11415244...
2        ((1208519184, 1208502800, 1208535568, 12085194...
3        ((776217216, 776217214, 776217218, 776216704, ...
4        ((1149584896, 1149568512, 1149601280, 11495846...
                               ...                        
19995    ((769662458, 769662456, 769662460, 769661946, ...
19996    ((767564038, 767564036, 767564040, 767563526, ...
19997    ((757249622, 757249620, 757249110, 757250134, ...
19998                                                   ()
19999    ((1141719040, 1141702656, 1141735424, 11417192...
Length: 20000, dtype: object

In [35]:
ev_tree["cluster_cell_ID"].array(entry_start=1,entry_stop=2, library="np")[0]   #Second event for numpy

<STLVector [[1141524224, 1141507840, 1141540608, ..., 757174864, 757175376, 757175378], ...] at 0x7fbcaa524520>

In [36]:
ev_tree["cluster_cell_ID"].array(library="pd")[1]                                #Second event for pandas

<STLVector [[1141524224, 1141507840, 1141540608, ..., 757174864, 757175376, 757175378], ...] at 0x7fbcb6f40970>

In [37]:
ev_tree['cluster_cell_ID'].array(entry_start=event_idx,entry_stop=event_idx+1,library='np')[0][0]   #Fist event for numpy, first cluster

<STLVector [759177244, 759177242, ..., 759177752, 754983430] at 0x7fbcb6f4a880>

In [100]:
### For each event: A vector per cluster with each cell value. 
cluster_cell_ID = ev_tree['cluster_cell_ID'].array(entry_start=event_idx,entry_stop=event_idx+1,library='np')[0]
cluster_cell_E = ev_tree['cluster_cell_E'].array(entry_start=event_idx,entry_stop=event_idx+1,library='np')[0]

In [62]:
id_to_position[759177244]

array([1643.8203 ,  614.2717 , -742.10706], dtype=float32)

In [39]:
n_clusters = len(cluster_cell_ID)
n_clusters

3

In [106]:
## ---- loop over clusters ---- ##
for ic in range(n_clusters) : 
    cell_E = np.array(cluster_cell_E[ic])
    cell_idx = np.array(cluster_cell_ID[ic])
    cluster_cell_pos = [id_to_position[x] for x in cell_idx]
    cluster_cell_deta = [id_to_deta[x] for x in cell_idx]
    cluster_cell_dphi = [id_to_dphi[x] for x in cell_idx]
    
    n_part = len(cluster_cell_pos)
    
    cluster_energy_truth = ev_tree['cluster_ENG_CALIB_TOT'].array(entry_start=event_idx,entry_stop=event_idx+1,library='np')[0][ic]
    
    if (ic==0):
        print("--------CLUSTER NUMBER: ",ic)
        print("Number of cells= ",len(cell_E))
        print("* Number of particles = Number of cells= ",len(cluster_cell_pos))
        print("* cell_E", cell_E)                                # Energy function?  Normalised to the total energy of the cluster?
        print("* cell_idx",cell_idx)
        print("* cluster_cell_pos",cluster_cell_pos)             # Angular function?
        print("* Total sum of cell energy= ", np.sum(cell_E))
        print("* ???????Cluster energy= ", ev_tree["cluster_E"].array(entry_start=event_idx,entry_stop=event_idx+1,library="np")[0][ic])
        print("* Cluster true energy= ", ev_tree['cluster_ENG_CALIB_TOT'].array(entry_start=event_idx,entry_stop=event_idx+1,library='np')[0][ic])

--------CLUSTER NUMBER:  0
Number of cells=  38
* Number of particles = Number of cells=  38
* cell_E [0.5656468  0.00999555 0.37462845 0.08905287 0.13298044 0.4048535
 0.05181599 0.06520326 0.07921552 0.02094988 0.03186893 0.01994124
 0.03106216 0.10201096 0.33802935 0.83388823 0.02306807 0.29424202
 0.02199198 0.01301084 0.00604178 0.02602894 0.03408835 0.07578252
 0.01392713 0.00503008 0.03000781 0.02797122 0.02698843 0.02797122
 0.02602894 0.01802212 0.10201096 0.0701307  0.03106216 0.0189655
 0.06092904 0.07578252]
* cell_idx [759177244 759177242 759177246 759176732 759177756 759176730 759177754
 759176734 759177758 757137414 757138438 757138950 757139462 757139974
 757140486 757140998 757140996 757141510 757140484 757141508 757141512
 759177240 759176728 759176218 759176216 759176220 757133830 757134342
 757134854 759177248 761270302 757139976 757142022 757142534 757143046
 757139464 759177752 754983430]
* cluster_cell_pos [array([1643.8203 ,  614.2717 , -742.10706], dtype=float3

In [84]:
ev_tree["cluster_E"].array(entry_start=event_idx,entry_stop=event_idx+1,library="np")[0][0]

3.9325354

In [89]:
cluster_cell_energy_0= ev_tree['cluster_cell_E'].array(entry_start=event_idx,entry_stop=event_idx+1,library='np')[0][0]
np.sum(cluster_cell_energy_0)

4.1802254

In [103]:
len(cell_idx)

31

In [72]:
print("?????Cell Energies in the last cluster", len(ev_tree['cluster_cell_E'].array()[0][2]))

?????Cell Energies in the last cluster 31
